In [1]:
modello_per_analisi = 'Opel_Corsa'
# Opzioni possibili: Mercedes_Classe_A, BMW_Serie_1, Audi_A3, Fiat_Panda

### 0. Import funzioni e librerie

In [2]:
import pandas as pd
import os
import sys
import requests
import importlib
from pathlib import Path


In [3]:
project_root = Path.cwd().parents[1]

# Percorsi delle cartelle contenenti i moduli
loader_dir = project_root / 'Source' / '2_Data_Preparation' / 'Utils'
utils_dir = project_root / 'Source' / '3_Data_Analysis' / 'Utils'

# Aggiungi al sys.path
sys.path.extend([str(loader_dir), str(utils_dir)])

# Import
from data_loader import load_csv_for_notebook
from Plots_maker import plot_is_conveniente_distribution, plot_price_prediction_distributions
from Function_price_analysis import modello_ml, predizione_prezzo, is_conveniente_class_report, indice_appetibilita, display_top_auto

In [4]:
# --- IMPORT CONFIGURAZIONE DELL'AUTO ---
#Configurazione path per import dinamico del modulo di configurazione 

config_path = project_root / 'Data'/ 'config'

sys.path.extend([str(config_path)])
print("\n - config_path:", config_path)
print(" - Contenuto della cartella:", os.listdir(config_path))

# --- Import dinamico del modulo di configurazione in base al modello ---
modulo_config = f"config_{modello_per_analisi}"  # esempio: config_Mercedes_Classe_A
try:
    config = importlib.import_module(modulo_config)
except ModuleNotFoundError:
    print(f"❌ Modulo di configurazione '{modulo_config}' non trovato.")
    sys.exit(1)
# --- Accesso ai parametri di configurazione ---
#allestimenti = config.allestimenti
allestimento_performance = config.allestimento_performance
allestimento_sport = config.allestimento_sport
allestimento_middle = config.allestimento_middle
allestimento_base = config.allestimento_base


 - config_path: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/config
 - Contenuto della cartella: ['config_Audi_A3.py', 'config_Opel_Corsa.py', '__pycache__', 'config_BMW_Serie_1.py', 'config_Mercedes_Classe_A.py']


### 1. Dati

In [5]:
#### CARICAMENTO DATI SINGOLI
data_dummy= load_csv_for_notebook(project_root,['Data', 'Processed_data', modello_per_analisi, f'data_dummy_{modello_per_analisi}.csv'])
data = load_csv_for_notebook(project_root,['Data', 'Processed_data', modello_per_analisi, f'data_{modello_per_analisi}.csv'])

X = data_dummy.drop(columns=['Prezzo', 'Distanza', 'Venditore'])
y = data_dummy['Prezzo']

✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/Opel_Corsa/data_dummy_Opel_Corsa.csv (4821 righe, 17 colonne)
✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/Opel_Corsa/data_Opel_Corsa.csv (4821 righe, 18 colonne)


### 2. Predizione prezzo, analisi della convenienza

In [6]:
modello_ml(X, y, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))
data_ml, y_pred = predizione_prezzo(data_dummy, X, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))

Valutazione delle performance sul test set
 - Mean Squared Error: 2268851.52
 - Root Mean Squared Error: 1506.27 euro
 - R-squared:  0.69


In [7]:
data_ml

,Prezzo,Anni,Chilometraggio,Cambio,CV,Venditore,Distanza,is_Diesel vs Benzina,is_GPL vs Benzina,is_Ibrido vs Benzina,is_Elettrico vs Benzina,is_sport vs base,is_performance vs base,is_middle vs base,is NO vs NE,is Centro vs NE,is Sud_Isole vs NE,prezzo_previsto,delta_prezzo,is_conveniente
0,4111,4,150000,0,102,1,172,1,0,0,0,0,0,1,0,0,0,6495.34,2384.34,1
1,6200,3,6000,0,102,1,119,0,0,0,0,0,0,0,0,0,0,8699.20,2499.20,1
2,7990,5,200000,0,102,0,471,1,0,0,0,1,0,0,0,0,1,9033.60,1043.60,1
3,7900,4,75000,0,102,1,299,0,0,0,0,0,0,0,1,0,0,8756.70,856.70,1
4,8990,4,118000,0,102,1,746,0,0,0,0,0,0,1,0,0,1,9412.88,422.88,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4816,17000,1,12000,0,75,0,99,0,0,0,0,1,0,0,0,1,0,15960.70,-1039.30,0
4817,14000,4,37000,0,75,0,861,0,0,0,0,0,0,1,0,0,1,12116.60,-1883.40,0
4818,12500,5,70000,0,75,0,141,0,0,0,0,0,0,0,0,0,0,12055.20,-444.80,0
4819,17500,2,12300,0,100,0,356,1,0,0,0,1,0,0,0,0,1,15444.35,-2055.65,0


In [8]:
plot_is_conveniente_distribution(data_ml)


Distribuzione relativa della variabile 'is_conveniente':

 is_conveniente  Frequenza Relativa
              0            0.620203
              1            0.379797


In [9]:
plot_price_prediction_distributions(data_ml)

In [10]:
#is_conveniente_class_report(data_ml)

### 3. Indice di attendibilità

In [11]:
#Prezzo soglia: prezzo oltre il quale il punteggio di attendibilità diminuisce
prezzo_soglia=35000

#Valori per il calcolo indice di attendibilità
valore_idx_anni = 0.025
valore_idx_prezzo = 0.15
valore_idx_km= 0.15 
valore_idx_dist = 0.10 
valore_idx_allestimento= 0.25 
valore_idx_cv =0.025
valore_idx_cambio = 0.20 

In [12]:
df = indice_appetibilita(data, y_pred, prezzo_soglia, valore_idx_anni, valore_idx_prezzo, valore_idx_km, 
                           valore_idx_dist, valore_idx_allestimento, valore_idx_cv, valore_idx_cambio, 
                           allestimento_performance, allestimento_sport,allestimento_middle,allestimento_base)

In [13]:
prezzo_max = 12000
prezzo_min = 7000
km_max =90000
km_min = 0
dist_max = 150
dispaly = 20

display_top_auto(df, dispaly, prezzo_max, prezzo_min, km_max, km_min, dist_max, carburante=None)


Le migliori 20 auto secondo l'indice di appetibilità sono:


,Annuncio,Link,Prezzo,Anni,Immatricolazione,Chilometraggio,Cambio,Carburante,CV,Allestimento,...,Venditore,Distanza,Comune,CAP,Regione,Area,Indice_Appetibilità,prezzo_previsto,delta_prezzo,is_conveniente
242,Opel Corsa Corsa VI 2020 e- GS Line,https://www.autoscout24.it/annunci/opel-corsa-...,11500,5,2020,58500,automatico,Benzina,102,GS,...,Privato,116,borgoricco,35010,Veneto,Nord-est,0.76,12320.50,820.50,1
223,Opel Corsa GS 6ª serie GS 1.2 *KM 0*,https://www.autoscout24.it/annunci/opel-corsa-...,11980,0,2025,12,manuale,Benzina,102,GS,...,Rivenditore,83,pistoia,51100,Toscana,Centro,0.63,12406.59,426.59,1
390,Opel Corsa GS 6ª serie GS 1.2 *KM 0*,https://www.autoscout24.it/annunci/opel-corsa-...,11980,0,2025,12,manuale,Benzina,102,GS,...,Rivenditore,82,montemurlo,59013,Toscana,Centro,0.63,12406.59,426.59,1
1859,OPEL Corsa 1.2 100 CV GS Line +,https://www.automobile.it/castenaso-opel-corsa...,9990,4,2021,76,manuale,Benzina,100,GS,...,Rivenditore,31,castenaso,40055,Emilia-Romagna,Nord-est,0.63,11249.64,1259.64,1
3444,Opel Corsa 1.2 100 CV GS Line +,https://autosupermarket.it/annunci/opel-corsa-...,9990,4,2021,76,manuale,Benzina,101,GS,...,Rivenditore,31,castenaso,40055,Emilia-Romagna,Nord-est,0.63,10961.46,971.46,1
1855,Opel Corsa 1.2 100 CV GS Line,https://www.automobile.it/certaldo-opel-corsa-...,9900,4,2021,68,manuale,Benzina,100,GS,...,Rivenditore,124,certaldo,50052,Toscana,Centro,0.62,10656.20,756.20,1
2108,Opel Corsa GS 1.2 *KM 0*,https://www.automobile.it/pistoia-opel-corsa-g...,11980,0,2025,12,manuale,Benzina,75,GS,...,Rivenditore,83,pistoia,51100,Toscana,Centro,0.62,13034.65,1054.65,1
3402,Opel Corsa 1.2 100 CV GS Line,https://autosupermarket.it/annunci/opel-corsa-...,11500,4,2021,82,manuale,Benzina,101,GS,...,Rivenditore,20,castel maggiore,40013,Emilia-Romagna,Nord-est,0.62,11422.02,-77.98,0
4019,Opel Corsa Gs 1.2 Km 0,https://www.subito.it/auto/opel-corsa-gs-1-2-k...,11980,0,2025,12,manuale,Benzina,75,GS,...,Privato,82,montemurlo,59013,Toscana,Centro,0.62,13034.65,1054.65,1
2022,Opel Corsa 1.5 diesel 100 CV GS Line,https://www.automobile.it/pieve-nievole-opel-c...,11500,5,2020,130,manuale,Diesel,100,GS,...,Rivenditore,91,pieve a nievole,51018,Toscana,Centro,0.61,11478.56,-21.44,0
